In [8]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['문서 분류', '유사 문장 생성']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess
text_dict = {}
#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        tmp_id = ''
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            text_id = line['id']
            text = line['Q_refined']
            department = line['labeling']['department']
            intent = line['labeling']['intent']
            category = intent['category']
            subcategory = intent['subcategory']
            predication = intent['predication']
            text_dict[text_id] = {'text':text, 'similar_Q':line['labeling']['similar_Q']}
            #### data preprocess end 

            if text_id.split('_')[0] != tmp_id:
                tmp_id = text_id.split('-')[0]
                #### 문서 분류
                data = {'input': None, 'output': {}}
                ## preprocess data from line
                data['input'] = text
                data['output'] = {'department': department, 
                                'category': category, 
                                'subcategory': subcategory, 
                                'predication': predication}
                ## preprocess data from line end
                task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### 문서 분류 end

    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            if line['labeling']['similar_Q'] == []:
                continue
            #### 유사 문장 생성
            data = {'input': None, 'output': None}
            ## preprocess data from line
            data['input'] = line['Q_refined']
            data['output'] = [text_dict[tid]['text'] for tid in line['labeling']['similar_Q'] if tid in text_dict]
            ## preprocess data from line end
            task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 유사 문장 생성 end


    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 1/1 [00:00<00:00,  1.27it/s]
